# AutoGen Grunnleggende Eksempel

I dette kodeeksempelet vil du bruke [AutoGen](https://aka.ms/ai-agents/autogen) AI-rammeverket for å lage en grunnleggende agent.

Målet med dette eksempelet er å vise deg trinnene som vi senere vil bruke i de ekstra kodeeksemplene når vi implementerer de forskjellige agentmønstrene.


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Opprett klienten

I dette eksempelet vil vi bruke [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for tilgang til LLM.

`model` er definert som `gpt-4o-mini`. Prøv å endre modellen til en annen modell som er tilgjengelig på GitHub Models-markedsplassen for å se forskjellige resultater.

Som en rask test vil vi bare kjøre en enkel forespørsel - `Hva er hovedstaden i Frankrike`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definere Agenten

Nå som vi har satt opp `client` og bekreftet at den fungerer, la oss opprette en `AssistantAgent`. Hver agent kan tildeles:  
**name** - Et kort navn som er nyttig for å referere til den i multi-agent flyter.  
**model_client** - Klienten du opprettet i det tidligere steget.  
**tools** - Tilgjengelige verktøy som agenten kan bruke for å fullføre en oppgave.  
**system_message** - Metaprompten som definerer oppgaven, oppførselen og tonen til LLM.  

Du kan endre systemmeldingen for å se hvordan LLM reagerer. Vi vil dekke `tools` i leksjon #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Kjør agenten

Funksjonen nedenfor vil kjøre agenten. Vi bruker metoden `on_message` for å oppdatere agentens tilstand med den nye meldingen.

I dette tilfellet oppdaterer vi tilstanden med en ny melding fra brukeren som er `"Planlegg en flott solfylt ferie"`.

Du kan endre innholdet i meldingen for å se hvordan LLM reagerer annerledes.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
